PA2 Notebook 2: Analysis
===

This is the second notebook for DRWI Pollution Assessment Stage 2 (PA2) analysis.

It reads dataframes prepared in Notebook 1 and calculates Pollution Assessment metrics necessary for the Stage 2 Assessment.

# Installation and Setup

Carefully follow our **[Installation Instructions](README.md#get-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependencies

In [1]:
from pathlib import Path
from importlib import reload

import pandas    as pd
import geopandas as gpd

# geo packages
from shapely.geometry import Polygon
import contextily as ctx

# packages for viz 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from  matplotlib.colors import LogNorm

In [2]:
# Confirm that this repo is in your Python Path
!conda-develop /Users/aaufdenkampe/Documents/Python/pollution-assessment/src

path exists, skipping /Users/aaufdenkampe/Documents/Python/pollution-assessment/src
completed operation for: /Users/aaufdenkampe/Documents/Python/pollution-assessment/src


In [3]:
# Custom functions for Pollution Assessment
import pollution_assessment as pa
import pollution_assessment.calc
import pollution_assessment.plot

In [4]:
# Confirm GeoPandas >= 0.11, for full GeoParquet support
print("Geopandas: ", gpd.__version__)

Geopandas:  0.11.1


## Set Paths and Open Files from Notebook 1


In [5]:
# Set your project directory to your local folder for your clone of this repository
project_path = Path.cwd().parent
project_path

PosixPath('/Users/aaufdenkampe/Documents/Python/pollution-assessment')

In [6]:
# Assign relative paths for the data OUTPUT folder.
pa2_data_output_folder = Path('stage2/data_output')

data_output_path = project_path /pa2_data_output_folder

In [7]:
%%time
# read geometry data from GeoParquet files
reach_gdf = gpd.read_parquet(data_output_path /'reach_gdf.parquet')
catch_gdf = gpd.read_parquet(data_output_path /'catch_gdf.parquet')

CPU times: user 1.1 s, sys: 114 ms, total: 1.21 s
Wall time: 1.22 s


In [8]:
%%time
# read results data from Parquet files
reach_concs_df = pd.read_parquet(data_output_path /'reach_concs_df.parquet')
catch_loads_df = pd.read_parquet(data_output_path /'catch_loads_df.parquet')

CPU times: user 78.8 ms, sys: 18.4 ms, total: 97.2 ms
Wall time: 41.4 ms


# Baseline Results
Create mappable GeoDataFrames (GDF) of results for "baseline" model runs (i.e. 'No restoration or protection')

In [9]:
# Use pollutant names dictionary
pa.calc.pollutants

{'TotalN': 'tn', 'TotalP': 'tp', 'Sediment': 'tss'}

In [10]:
# Create Stream Reach Concentrations GDF
reach_concs_gdf = pa.calc.join_results(
    'reach', reach_gdf, reach_concs_df, 
    pa.calc.run_groups[0], ps=False
)
# Add columns with previously used short names
for pollutant in pa.calc.pollutants.keys():
    reach_concs_gdf [f'{pa.calc.pollutants[pollutant]}_conc'] = (
        reach_concs_gdf[f'{pollutant}'])

# reach_concs_gdf.info()

In [11]:
# Create Catchment Loads GDF
catch_loads_gdf = pa.calc.join_results(
    'catch', catch_gdf, catch_loads_df, 
    pa.calc.run_groups[0], ps=False
)
pa.calc.add_loadrate(catch_loads_gdf)
catch_loads_gdf.head()

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,Source,Sediment,TotalN,TotalP,run_group,funding_sources,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,Total Local Load,1.101612e+06,12680.544786,1189.608231,No restoration or protection,NaN,True,1.951842,0.183109,169.564734
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,Total Local Load,2.013339e+05,3771.332143,363.366436,No restoration or protection,NaN,True,2.267555,0.218478,121.054237
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,Total Local Load,2.514024e+05,3133.430355,357.227990,No restoration or protection,NaN,True,1.911313,0.217900,153.349047
1748541,3013.8348,12912.30,86.528,"MULTIPOLYGON (((-8309824.403 5193427.492, -830...",drb,<NA>,74911,74915,020401020302,2,...,Total Local Load,6.177144e+05,6409.514442,668.969079,No restoration or protection,NaN,True,2.126697,0.221966,204.959595
1748543,1151.0990,5232.87,35.389,"MULTIPOLYGON (((-8312514.529 5185023.831, -831...",drb,<NA>,74920,74922,020401020305,2,...,Total Local Load,1.989546e+05,2918.236825,317.461447,No restoration or protection,NaN,True,2.535174,0.275790,172.838845


# XS: Excess Pollution Results

```
excess pollution = total pollution 
                   – threshold pollution target
```

In [12]:
# Open dictionary of Target Values
pa.calc.targets

{'tn': {'loadrate_target': 17.07, 'conc_target': 4.73},
 'tp': {'loadrate_target': 0.31, 'conc_target': 0.09},
 'tss': {'loadrate_target': 923.8, 'conc_target': 237.3}}

In [13]:
pa.calc.add_excess('reach', reach_concs_gdf)
reach_concs_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,TotalP,run_group,funding_sources,with_attenuation,tn_conc,tp_conc,tss_conc,tn_conc_xs,tp_conc_xs,tss_conc_xs
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,MULTILINESTRING Z ((-8295323.930 5214456.622 0...,drb,<NA>,74914,74914,020401020302,1,...,0.030464,No restoration or protection,NaN,True,0.324727,0.030464,28.210388,-4.405273,-0.059536,-209.089612
1748537,1663.1712,1664.46,11.189,MULTILINESTRING Z ((-8304623.226 5207684.737 0...,drb,<NA>,74913,74913,020401020302,1,...,0.036342,No restoration or protection,NaN,True,0.377186,0.036342,20.136201,-4.352814,-0.053658,-217.163799
1748539,1639.4128,1640.70,11.223,MULTILINESTRING Z ((-8316446.558 5197994.113 0...,drb,<NA>,74921,74921,020401020305,1,...,0.035620,No restoration or protection,NaN,True,0.312437,0.035620,25.067574,-4.417563,-0.054380,-212.232426


In [14]:
pa.calc.add_excess('catch', catch_loads_gdf)
catch_loads_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,TotalP,run_group,funding_sources,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate,tn_loadrate_xs,tp_loadrate_xs,tss_loadrate_xs
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,1189.608231,No restoration or protection,NaN,True,1.951842,0.183109,169.564734,-15.118158,-0.126891,-754.235266
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,363.366436,No restoration or protection,NaN,True,2.267555,0.218478,121.054237,-14.802445,-0.091522,-802.745763
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,357.227990,No restoration or protection,NaN,True,1.911313,0.217900,153.349047,-15.158687,-0.092100,-770.450953


# XSNPS: Excess Non-Point Source Results

```
excess nonpoint source pollution = excess pollution 
                                   – point source pollution
```

In [15]:
pa.calc.add_xsnps('reach', reach_concs_gdf, reach_concs_df)
reach_concs_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,with_attenuation,tn_conc,tp_conc,tss_conc,tn_conc_xs,tp_conc_xs,tss_conc_xs,tn_conc_xsnps,tp_conc_xsnps,tss_conc_xsnps
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,MULTILINESTRING Z ((-8295323.930 5214456.622 0...,drb,<NA>,74914,74914,020401020302,1,...,True,0.324727,0.030464,28.210388,-4.405273,-0.059536,-209.089612,-4.405273,-0.059536,-209.089612
1748537,1663.1712,1664.46,11.189,MULTILINESTRING Z ((-8304623.226 5207684.737 0...,drb,<NA>,74913,74913,020401020302,1,...,True,0.377186,0.036342,20.136201,-4.352814,-0.053658,-217.163799,-4.352814,-0.053658,-217.163799
1748539,1639.4128,1640.70,11.223,MULTILINESTRING Z ((-8316446.558 5197994.113 0...,drb,<NA>,74921,74921,020401020305,1,...,True,0.312437,0.035620,25.067574,-4.417563,-0.054380,-212.232426,-4.417563,-0.054380,-212.232426


In [16]:
pa.calc.add_xsnps('catch', catch_loads_gdf, catch_loads_df)
catch_loads_gdf.head(3)

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,with_attenuation,tn_loadrate,tp_loadrate,tss_loadrate,tn_loadrate_xs,tp_loadrate_xs,tss_loadrate_xs,tn_loadrate_xsnps,tp_loadrate_xsnps,tss_loadrate_xsnps
comid,,,,,,,,,,,,,,,,,,,,,
1748535,6496.7052,6501.69,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",drb,<NA>,74914,74914,020401020302,1,...,True,1.951842,0.183109,169.564734,-15.118158,-0.126891,-754.235266,-15.118158,-0.126891,-754.235266
1748537,1663.1712,1664.46,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",drb,<NA>,74913,74913,020401020302,1,...,True,2.267555,0.218478,121.054237,-14.802445,-0.091522,-802.745763,-14.802445,-0.091522,-802.745763
1748539,1639.4128,1640.70,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",drb,<NA>,74921,74921,020401020305,1,...,True,1.911313,0.217900,153.349047,-15.158687,-0.092100,-770.450953,-15.158687,-0.092100,-770.450953


## Confirm Results vs. MMW Sub-basin

Confirm values make sense for COMIDs with:
- no point sources: `4648450`
- large point sources: `932040160`

In [17]:
reach_concs_gdf[['tp_conc',
                'tp_conc_xs',
                'tp_conc_xsnps',
               ]].loc[[4648450, 932040160]]

,tp_conc,tp_conc_xs,tp_conc_xsnps
comid,,,
4648450,0.324502,0.234502,0.234502
932040160,0.284033,0.194033,0.107405


In [18]:
catch_loads_gdf[['tp_loadrate',
                'tp_loadrate_xs',
                'tp_loadrate_xsnps',
               ]].loc[[4648450, 932040160]]

,tp_loadrate,tp_loadrate_xs,tp_loadrate_xsnps
comid,,,
4648450,1.554258,1.244258,1.244258
932040160,7.054176,6.744176,0.187592


In [19]:
catch_loads_gdf[['tn_loadrate',
                'tn_loadrate_xs',
                'tn_loadrate_xsnps',
               ]].loc[[4648450, 932040160]]

,tn_loadrate,tn_loadrate_xs,tn_loadrate_xsnps
comid,,,
4648450,25.356569,8.286569,8.286569
932040160,49.770205,32.700205,-8.232529


In [20]:
df = pa.calc.calc_loadrate(catch_loads_gdf, catch_loads_df, 
                            'TotalN', 
                            pa.calc.run_groups[0], ps=True)
df.loc[[4648450, 932040160]]

comid
4648450       0.000000
932040160    40.932734
dtype: float64

# REM: Remaining XSNPS after Restoration

In [21]:
for group_key in [1,2,3]:
    pa.calc.add_remaining('reach', reach_concs_gdf, reach_concs_df, group_key)
reach_concs_gdf.loc[[4648450, 932040160]]

,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_conc_xsnps,tn_conc_rem1,tp_conc_rem1,tss_conc_rem1,tn_conc_rem2,tp_conc_rem2,tss_conc_rem2,tn_conc_rem3,tp_conc_rem3,tss_conc_rem3
comid,,,,,,,,,,,,,,,,,,,,,
4648450,263.4373,263.61,1.412,MULTILINESTRING Z ((-8449613.219 4882948.059 0...,Brandywine and Christina,<NA>,64639,64639,020402050202,1,...,-86.572683,0.249977,0.062610,-112.257291,0.249977,0.062610,-112.257291,0.248132,0.059352,-112.762403
932040160,497.8113,14124.96,74.903,MULTILINESTRING Z ((-8440707.897 4862261.408 0...,Brandywine and Christina,<NA>,64609,64658,020402050202,3,...,-96.816710,0.066303,0.143535,-193.251167,0.066303,0.143535,-193.251167,0.065058,0.141779,-193.573524


In [22]:
for group_key in [1,2,3]:
    pa.calc.add_remaining('catch', catch_loads_gdf, catch_loads_df, group_key)
catch_loads_gdf.loc[[4648450, 932040160]]

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_loadrate_xsnps,tn_loadrate_rem1,tp_loadrate_rem1,tss_loadrate_rem1,tn_loadrate_rem2,tp_loadrate_rem2,tss_loadrate_rem2,tn_loadrate_rem3,tp_loadrate_rem3,tss_loadrate_rem3
comid,,,,,,,,,,,,,,,,,,,,,
4648450,263.4373,263.61,1.412,"MULTIPOLYGON (((-8449229.677 4880762.406, -844...",Brandywine and Christina,<NA>,64639,64639,020402050202,1,...,-201.865928,6.782443,0.420951,-324.886720,6.782443,0.420951,-324.886720,6.773609,0.405346,-327.306039
932040160,497.8113,14124.96,74.903,"MULTIPOLYGON (((-8442711.700 4860467.546, -844...",Brandywine and Christina,<NA>,64609,64658,020402050202,3,...,162.418203,32.700205,6.744176,162.418203,32.700205,6.744176,162.418203,32.698994,6.742603,162.114027


# AVOID: Avoided Loads due to Protection

In [23]:
pa.calc.run_groups[4]

'Direct WPF Protection'

In [24]:
pa.calc.add_avoided('reach', reach_concs_gdf, reach_concs_df, 4)
reach_concs_gdf.loc[pa.calc.comid_test_list,
    ['tp_conc', 'tp_conc_xsnps','tp_conc_rem3', 'tp_conc_avoid']]

,tp_conc,tp_conc_xsnps,tp_conc_rem3,tp_conc_avoid
comid,,,,
4648450,0.324502,0.234502,0.059352,0.000000
932040160,0.284033,0.107405,0.141779,0.000000
2583195,0.064066,-0.025934,-0.025940,0.001256
932040230,0.006167,-0.083833,-0.083833,0.000514


In [25]:
pa.calc.add_avoided('catch', catch_loads_gdf, catch_loads_df, 4)
catch_loads_gdf.loc[pa.calc.comid_test_list,
    ['tp_loadrate', 'tp_loadrate_xsnps','tp_loadrate_rem3', 'tp_loadrate_avoid']]

,tp_loadrate,tp_loadrate_xsnps,tp_loadrate_rem3,tp_loadrate_avoid
comid,,,,
4648450,1.554258,1.244258,0.405346,0.000000
932040160,7.054176,0.187592,6.742603,0.000000
2583195,0.359205,0.049205,0.049172,0.007041
932040230,0.082214,-0.227786,-0.227786,0.007001


In [26]:
catch_loads_gdf.loc[(catch_loads_gdf.tp_loadrate_avoid > 0)]

,catchment_hectares,watershed_hectares,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,...,tss_loadrate_rem1,tn_loadrate_rem2,tp_loadrate_rem2,tss_loadrate_rem2,tn_loadrate_rem3,tp_loadrate_rem3,tss_loadrate_rem3,tn_loadrate_avoid,tp_loadrate_avoid,tss_loadrate_avoid
comid,,,,,,,,,,,,,,,,,,,,,
2583191,277.9706,356.76,2.257,"MULTIPOLYGON (((-8321373.063 5036691.907, -832...",New Jersey Highlands,<NA>,70084,70085,020401050101,1,...,-645.705971,-14.543512,-0.070038,-645.705971,-14.543590,-0.070066,-645.721320,0.000034,0.000305,0.012710
2583195,244.9663,245.16,1.537,"MULTIPOLYGON (((-8322884.353 5034593.476, -832...",New Jersey Highlands,<NA>,70082,70082,020401050101,1,...,-492.602728,-13.993447,0.049205,-492.602728,-13.993536,0.049172,-492.620739,0.000776,0.007041,0.292851
2583199,215.1096,1018.44,7.164,"MULTIPOLYGON (((-8325931.539 5035114.436, -832...",New Jersey Highlands,<NA>,70074,70080,020401050101,2,...,-444.071676,-15.198850,-0.088779,-444.071676,-15.199058,-0.088856,-444.112843,0.000358,0.003214,0.134492
2583463,163.6690,163.80,1.131,"MULTIPOLYGON (((-8348195.342 5014278.655, -834...",New Jersey Highlands,<NA>,70175,70175,020401050104,1,...,-733.990121,-16.267772,-0.227914,-733.990121,-16.267783,-0.227928,-733.990889,0.000018,0.000164,0.007471
2583501,343.6150,1712.34,13.092,"MULTIPOLYGON (((-8349076.358 5012570.234, -834...",New Jersey Highlands,<NA>,70168,70175,020401050104,2,...,-762.747681,-16.007439,-0.221786,-762.747681,-16.007477,-0.221847,-762.749445,0.000030,0.000273,0.012480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040230,21.4928,667.08,5.065,"MULTIPOLYGON (((-8357072.635 5048834.448, -835...",Poconos and Kittatinny,<NA>,70857,70860,020401040602,1,...,-711.287136,-16.552876,-0.227786,-711.287136,-16.552876,-0.227786,-711.287136,0.000791,0.007001,0.327289
932040231,271.7625,368.64,2.541,"MULTIPOLYGON (((-8354173.968 5049819.487, -835...",Poconos and Kittatinny,<NA>,70859,70860,020401040602,1,...,-847.183465,-16.630928,-0.270726,-847.183465,-16.630928,-0.270726,-847.183465,0.000296,0.002561,0.121161
932040232,276.7087,645.57,4.902,"MULTIPOLYGON (((-8356477.168 5048011.609, -835...",Poconos and Kittatinny,<NA>,70858,70860,020401040602,1,...,-828.849766,-16.075971,-0.255903,-828.849766,-16.075972,-0.255904,-828.849952,0.000734,0.006189,0.297029


In [27]:
# Reload module to apply any new edits to functions
reload(pa.calc)

<module 'pollution_assessment.calc' from '/Users/aaufdenkampe/Documents/Python/pollution-assessment/src/pollution_assessment/calc.py'>

# Save Calculated PA2 Results

In [29]:
%%time
# Save PA2 combined and calculated results
reach_concs_gdf.to_parquet(data_output_path /'reach_concs_gdf.parquet',compression='gzip')
catch_loads_gdf.to_parquet(data_output_path /'catch_loads_gdf.parquet',compression='gzip')

CPU times: user 4.4 s, sys: 90.7 ms, total: 4.49 s
Wall time: 4.59 s
